In [1]:
print('hello world')

hello world


In [6]:
from datetime import datetime,timedelta

In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time, pandas as pd
import os

# Setup Selenium
options = Options()
options.add_argument("--lang=id")
options.add_argument("accept-language=id-ID,id")
options.add_argument("--start-maximized")
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless')  
options.add_argument('--disable-gpu')

driver = webdriver.Chrome(options=options)

# URL lokasi Google Maps
driver.get("https://www.google.com/maps/place/Kantor+Pos+Fatmawati/@-6.2945288,106.7947741,17z/data=!4m16!1m9!3m8!1s0x2e69f19684fb0151:0xb0b69f82cb32571a!2sKantor+Pos+Fatmawati!8m2!3d-6.2945288!4d106.7947741!9m1!1b1!16s%2Fg%2F11dfj2fyrd!3m5!1s0x2e69f19684fb0151:0xb0b69f82cb32571a!8m2!3d-6.2945288!4d106.7947741!16s%2Fg%2F11dfj2fyrd?entry=ttu&g_ep=EgoyMDI1MDgwNS4wIKXMDSoASAFQAw%3D%3D")
time.sleep(5)

# Ambil nama tempat
try:
    place_name = driver.find_element(By.CLASS_NAME, "DUwDvf").text.strip()
except:
    place_name = "Unknown"

# Klik tombol "Ulasan lainnya"
try:
    review_button = driver.find_element(
        By.XPATH, "//span[contains(text(), 'Ulasan lainnya') or contains(text(), 'Lainnya')]/ancestor::button"
    )
    driver.execute_script("arguments[0].click();", review_button)
    time.sleep(8)
except:
    print("Tombol ulasan tidak ditemukan.")

# Scroll & ambil container ulasan
try:
    scrollable_div = driver.find_element(By.CSS_SELECTOR, 'div.m6QErb.DxyBCb.kA9KIf.dS8AEf')
except:
    print("Gagal menemukan container ulasan")
    driver.quit()
    exit()

reviews_list, seen = [], set()
scroll_count = 0
max_scrolls = 40

while len(reviews_list) < 500 and scroll_count < max_scrolls:
    elements = driver.find_elements(By.CSS_SELECTOR, "div.jftiEf")
    print(f"Scroll ke-{scroll_count+1}: {len(elements)} ulasan ditemukan.")

    # Klik semua tombol "Lainnya"
    try:
        more_buttons = driver.find_elements(By.CSS_SELECTOR, 'button.w8nwRe.kyuRq')
        for btn in more_buttons:
            try:
                driver.execute_script("arguments[0].click();", btn)
                time.sleep(0.2)
            except:
                continue
    except:
        pass

    # Ambil data ulasan
    for c in elements:
        try:
            user_el = c.find_element(By.CLASS_NAME, "d4r55")
            review_el = c.find_element(By.CLASS_NAME, "wiI7pd")
            rating_el = c.find_element(By.CLASS_NAME, "kvMYJc")
            date_el = c.find_element(By.CLASS_NAME, "rsqaWe")

            if not review_el or not review_el.text.strip():
                continue

            user = user_el.text.strip() if user_el else "Tidak ditemukan"
            review = review_el.text.strip()
            rating = rating_el.get_attribute("aria-label").split()[0] if rating_el else "Tidak ditemukan"
            date = date_el.text.strip() if date_el else "Tidak ditemukan"

            if (user, review) not in seen:
                seen.add((user, review))
                reviews_list.append({
                    "Pos Office Branch": place_name,
                    "User": user,
                    "Reviews": review,
                    "Rate": rating,
                    "Date": date,
                    'tanggal_scrap':datetime.now()
                })
        except:
            continue

    # Scroll ke bawah
    try:
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
    except Exception as e:
        print("Gagal scroll:", e)
        break

    scroll_count += 1
    time.sleep(2)


if not os.path.exists("data"):
    os.makedirs("data")

df_maps = pd.DataFrame(reviews_list)
# df_maps.to_csv("data/ULASAN_POS_JAKARTAFLORA.csv", index=False, encoding="utf-8-sig")
# print(f'\n✅ DONE: {len(reviews_list)} ulasan berhasil disimpan.')

driver.quit()

Scroll ke-1: 0 ulasan ditemukan.
Scroll ke-2: 10 ulasan ditemukan.
Scroll ke-3: 20 ulasan ditemukan.
Scroll ke-4: 30 ulasan ditemukan.
Scroll ke-5: 40 ulasan ditemukan.
Scroll ke-6: 50 ulasan ditemukan.
Scroll ke-7: 60 ulasan ditemukan.
Scroll ke-8: 70 ulasan ditemukan.
Scroll ke-9: 80 ulasan ditemukan.
Scroll ke-10: 90 ulasan ditemukan.
Scroll ke-11: 100 ulasan ditemukan.
Scroll ke-12: 110 ulasan ditemukan.
Scroll ke-13: 120 ulasan ditemukan.
Scroll ke-14: 130 ulasan ditemukan.
Scroll ke-15: 140 ulasan ditemukan.
Scroll ke-16: 150 ulasan ditemukan.
Scroll ke-17: 160 ulasan ditemukan.
Scroll ke-18: 170 ulasan ditemukan.
Scroll ke-19: 180 ulasan ditemukan.
Scroll ke-20: 190 ulasan ditemukan.
Scroll ke-21: 200 ulasan ditemukan.
Scroll ke-22: 210 ulasan ditemukan.
Scroll ke-23: 220 ulasan ditemukan.
Scroll ke-24: 230 ulasan ditemukan.
Scroll ke-25: 240 ulasan ditemukan.
Scroll ke-26: 250 ulasan ditemukan.
Scroll ke-27: 260 ulasan ditemukan.
Scroll ke-28: 270 ulasan ditemukan.
Scroll ke-29

In [38]:
df_maps.info()
df_maps.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Pos Office Branch  290 non-null    object        
 1   User               290 non-null    object        
 2   Reviews            290 non-null    object        
 3   Rate               290 non-null    object        
 4   Date               290 non-null    object        
 5   tanggal_scrap      290 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 13.7+ KB


,Pos Office Branch,User,Reviews,Rate,Date,tanggal_scrap
0,Kantor Pos Fatmawati,Fahmi Hidayat,Good job.\nPertama kali ke sini buat ngambil k...,5,3 minggu lalu,2025-08-11 11:27:11.057364
1,Kantor Pos Fatmawati,Ang,Pengalaman kirim barang di cabang pos ini cuku...,3,2 bulan lalu,2025-08-11 11:27:11.106878
2,Kantor Pos Fatmawati,Twitter [at] PapaEK0,Tempat nyaman dan strategis karena di lalui an...,5,Diedit seminggu lalu,2025-08-11 11:27:11.161409
3,Kantor Pos Fatmawati,Syafiq Sultonassydiq,"Ruang tunggu luas, parkiran luas. Kantor cende...",4,11 bulan lalu,2025-08-11 11:27:11.214207
4,Kantor Pos Fatmawati,ELIKA AINI,saya pemesanan pick up via aplikasi pos aja tu...,1,sebulan lalu,2025-08-11 11:27:11.263899


engineering data

In [39]:
df_maps.drop_duplicates(keep='first',inplace=True)

huruf kecil

In [40]:
for i in df_maps.columns:
    if df_maps[i].dtype=='object':
        df_maps[i]=df_maps[i].str.lower()

hapus extra space

In [41]:
df_maps['Date']=df_maps['Date'].str.replace('diedit','')

In [42]:
inidict={
    'sebulan':'1 bulan',
    'setahun':'1 tahun',
    'sehari': '1 hari',
    'seminggu': '1 minggu'
}

In [43]:
type(inidict),len(inidict),inidict

(dict,
 4,
 {'sebulan': '1 bulan',
  'setahun': '1 tahun',
  'sehari': '1 hari',
  'seminggu': '1 minggu'})

hapus kata tidak perlu

In [44]:
df_maps['Date']=df_maps['Date'].str.replace('lalu','')

hapus extra space

In [45]:
df_maps['Date']=df_maps['Date'].str.strip()

In [46]:
df_maps['Date'] = df_maps['Date'].map(inidict).fillna(df_maps['Date'])

In [47]:
df_maps['Date'].value_counts()

Date
5 tahun     40
6 tahun     33
1 tahun     31
4 tahun     31
2 tahun     29
3 tahun     25
7 tahun     24
8 tahun     21
9 tahun     10
3 minggu     6
11 bulan     6
1 bulan      6
3 hari       4
7 bulan      4
2 minggu     4
2 hari       2
5 bulan      2
9 bulan      2
2 bulan      2
4 bulan      1
8 bulan      1
1 minggu     1
4 minggu     1
6 bulan      1
3 bulan      1
10 tahun     1
1 hari       1
Name: count, dtype: int64

In [48]:
df_maps['waktu_dalam_angka']=df_maps['Date'].str.split(' ').str[0]

In [49]:
df_maps['waktu_dalam_ket']=df_maps['Date'].str.split(' ').str[1]

In [50]:
df_maps

,Pos Office Branch,User,Reviews,Rate,Date,tanggal_scrap,waktu_dalam_angka,waktu_dalam_ket
0,kantor pos fatmawati,fahmi hidayat,good job.\npertama kali ke sini buat ngambil k...,5,3 minggu,2025-08-11 11:27:11.057364,3,minggu
1,kantor pos fatmawati,ang,pengalaman kirim barang di cabang pos ini cuku...,3,2 bulan,2025-08-11 11:27:11.106878,2,bulan
2,kantor pos fatmawati,twitter [at] papaek0,tempat nyaman dan strategis karena di lalui an...,5,1 minggu,2025-08-11 11:27:11.161409,1,minggu
3,kantor pos fatmawati,syafiq sultonassydiq,"ruang tunggu luas, parkiran luas. kantor cende...",4,11 bulan,2025-08-11 11:27:11.214207,11,bulan
4,kantor pos fatmawati,elika aini,saya pemesanan pick up via aplikasi pos aja tu...,1,1 bulan,2025-08-11 11:27:11.263899,1,bulan
...,...,...,...,...,...,...,...,...
285,kantor pos fatmawati,fazli umar faruq,warisan,5,7 tahun,2025-08-11 11:41:30.071597,7,tahun
286,kantor pos fatmawati,rizwan hamdi,tempat yang bagus,5,5 tahun,2025-08-11 11:41:30.255276,5,tahun
287,kantor pos fatmawati,deercali,bagus,5,2 tahun,2025-08-11 11:42:22.912776,2,tahun
288,kantor pos fatmawati,putri,kerja bagus,5,3 hari,2025-08-11 11:42:23.170529,3,hari


In [51]:
from datetime import datetime,timedelta
import numpy as np

In [52]:
df_maps['waktu_dalam_ket'].value_counts()

waktu_dalam_ket
tahun     245
bulan      26
minggu     12
hari        7
Name: count, dtype: int64

In [53]:
df_maps

,Pos Office Branch,User,Reviews,Rate,Date,tanggal_scrap,waktu_dalam_angka,waktu_dalam_ket
0,kantor pos fatmawati,fahmi hidayat,good job.\npertama kali ke sini buat ngambil k...,5,3 minggu,2025-08-11 11:27:11.057364,3,minggu
1,kantor pos fatmawati,ang,pengalaman kirim barang di cabang pos ini cuku...,3,2 bulan,2025-08-11 11:27:11.106878,2,bulan
2,kantor pos fatmawati,twitter [at] papaek0,tempat nyaman dan strategis karena di lalui an...,5,1 minggu,2025-08-11 11:27:11.161409,1,minggu
3,kantor pos fatmawati,syafiq sultonassydiq,"ruang tunggu luas, parkiran luas. kantor cende...",4,11 bulan,2025-08-11 11:27:11.214207,11,bulan
4,kantor pos fatmawati,elika aini,saya pemesanan pick up via aplikasi pos aja tu...,1,1 bulan,2025-08-11 11:27:11.263899,1,bulan
...,...,...,...,...,...,...,...,...
285,kantor pos fatmawati,fazli umar faruq,warisan,5,7 tahun,2025-08-11 11:41:30.071597,7,tahun
286,kantor pos fatmawati,rizwan hamdi,tempat yang bagus,5,5 tahun,2025-08-11 11:41:30.255276,5,tahun
287,kantor pos fatmawati,deercali,bagus,5,2 tahun,2025-08-11 11:42:22.912776,2,tahun
288,kantor pos fatmawati,putri,kerja bagus,5,3 hari,2025-08-11 11:42:23.170529,3,hari


In [54]:
hariini=datetime.today().date()

In [55]:
hariini

datetime.date(2025, 8, 11)

In [56]:
df_maps['waktu_dalam_angka']=df_maps['waktu_dalam_angka'].astype(int)

In [57]:
from dateutil.relativedelta import relativedelta

In [58]:
def hitung_tanggal(row):
    angka = row['waktu_dalam_angka']
    ket = row['waktu_dalam_ket']
    if ket == 'tahun':
        return hariini - relativedelta(years=angka)
    elif ket == 'bulan':
        return hariini - relativedelta(months=angka)
    elif ket == 'minggu':
        return hariini - relativedelta(weeks=angka)
    elif ket == 'hari':
        return hariini - relativedelta(days=angka)
    else:
        return hariini

df_maps['tanggal_review'] = df_maps.apply(hitung_tanggal, axis=1)

In [59]:
df_maps.drop(columns=['Date','waktu_dalam_angka','waktu_dalam_ket'],inplace=True)

In [60]:
df_maps

,Pos Office Branch,User,Reviews,Rate,tanggal_scrap,tanggal_review
0,kantor pos fatmawati,fahmi hidayat,good job.\npertama kali ke sini buat ngambil k...,5,2025-08-11 11:27:11.057364,2025-07-21
1,kantor pos fatmawati,ang,pengalaman kirim barang di cabang pos ini cuku...,3,2025-08-11 11:27:11.106878,2025-06-11
2,kantor pos fatmawati,twitter [at] papaek0,tempat nyaman dan strategis karena di lalui an...,5,2025-08-11 11:27:11.161409,2025-08-04
3,kantor pos fatmawati,syafiq sultonassydiq,"ruang tunggu luas, parkiran luas. kantor cende...",4,2025-08-11 11:27:11.214207,2024-09-11
4,kantor pos fatmawati,elika aini,saya pemesanan pick up via aplikasi pos aja tu...,1,2025-08-11 11:27:11.263899,2025-07-11
...,...,...,...,...,...,...
285,kantor pos fatmawati,fazli umar faruq,warisan,5,2025-08-11 11:41:30.071597,2018-08-11
286,kantor pos fatmawati,rizwan hamdi,tempat yang bagus,5,2025-08-11 11:41:30.255276,2020-08-11
287,kantor pos fatmawati,deercali,bagus,5,2025-08-11 11:42:22.912776,2023-08-11
288,kantor pos fatmawati,putri,kerja bagus,5,2025-08-11 11:42:23.170529,2025-08-08
